In [7]:
import requests

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/202405-citibike-tripdata/202405-citibike-tripdata_1.csv')

/var/folders/st/m0b_93g97bxcmqnsx5v2qdm00000gn/T/ipykernel_6967/4292729557.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/202405-citibike-tripdata/202405-citibike-tripdata_1.csv')


In [3]:
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [4]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,FE6A074FC945A086,electric_bike,2024-05-01 08:05:53,2024-05-01 08:15:14,1 Ave & E 30 St,6079.03,E 74 St & 1 Ave,6953.08,40.741509,-73.975355,40.768974,-73.954823,member
1,6844CF42F72827D7,classic_bike,2024-05-10 15:00:06,2024-05-10 15:08:58,31 Ave & 57 St,6621.06,31 Ave & Steinway St,6782.01,40.757357,-73.904726,40.761149,-73.917007,member
2,D7C9E007F791A7A2,classic_bike,2024-05-06 12:28:39,2024-05-06 12:35:21,E 89 St & 3 Ave,7265.10,E 80 St & 2 Ave,7121.02,40.780628,-73.952167,40.773914,-73.954395,member
3,3419650E2F5999F1,classic_bike,2024-05-16 16:09:14,2024-05-16 16:14:20,E 89 St & 3 Ave,7265.10,E 80 St & 2 Ave,7121.02,40.780628,-73.952167,40.773914,-73.954395,member
4,5469FDC75E0F1FCA,electric_bike,2024-05-29 15:43:37,2024-05-29 15:49:27,1 Ave & E 30 St,6079.03,E 45 St & 3 Ave,6464.08,40.741444,-73.975361,40.752554,-73.972826,member


In [5]:
pd.__version__

'2.2.2'

In [8]:
# get the stations information

req = requests.get("https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json")

In [11]:
pd.DataFrame(req.json()['data']['stations'])

,short_name,lon,region_id,lat,capacity,rental_uris,station_id,name
0,5569.07,-74.003140,71,40.723230,51,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",7d5fa1f0-4069-4d5d-b735-5d500597394a,W Broadway & Watts St
1,4962.02,-74.013181,71,40.703662,1,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",c1a4d909-0a00-475a-8e82-18ed13a4eb01,Whitehall St & Bridge St
2,6098.12,-73.989873,71,40.738661,65,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",40043e8a-6ef6-428d-b585-a7c23cdb06d4,Broadway & E 19 St
3,7913.15,-73.908351,71,40.819890,19,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",19b61564-7629-41b4-80c8-1756135a5442,Cauldwell Ave & E 158 St
4,4962.08,-74.011678,71,40.703652,49,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",66db447a-0aca-11e7-82f6-3863bb44ef7c,Broad St & Bridge St
...,...,...,...,...,...,...,...,...
2211,JC105,-74.046964,70,40.735208,32,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",5faf99b8-9046-450f-9d2a-d13279b3d016,Hoboken Ave at Monmouth St
2212,JC093,-74.071959,70,40.725726,18,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",5aa81a37-ea1d-4bf6-ab40-4751c6b4fb08,Fairmount Ave
2213,HB603,-74.028199,311,40.745984,21,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",519824e4-69ba-4270-a395-17c204f328f8,8 St & Washington St
2214,JC027,-74.045572,70,40.725289,14,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",66ddd93e-0aca-11e7-82f6-3863bb44ef7c,Jersey & 6th St
